# CIS 545 - Big Data Analytics - Fall 2019

# Homework 2: Querying Linked (LinkedIn) Data

# Due October 11, 2019 at 10pm

Have you ever wondered about (1) what it takes to be a data scientist or "data person", and (2) how social networks and recommender systems work?

This homework is focused on (1) working with hierarchical data stored in dataframes, (2) traversing relationships among data, including graph data, (3) understanding a bit about performance.

We will focus on questions about data scientists from "our" crawl of the LinkedIn dataset, which was also used in the Lecture (module) 2 extended notebook.

In [0]:
!pip install pymongo[tls,srv]
!pip install swifter
!pip install lxml

In [0]:
import pandas as pd
import numpy as np
import json
import sqlite3
from lxml import etree
import urllib
import zipfile

import time
import swifter
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

# Step 0: Acquire and load data

We need to pull the zipfile with LinkedIn data from Amazon S3 (where it is shared) to your local machine or the Google Colab cloud-hosted machine.  Only when the data is local can we efficiently parse it (and we'll read directly out of a zip file).

The zip file contains three files with the same schema.  You can start with the `tiny` instance to test your queries, then go on to `small`.  If you're brave and have a lot of time feel free to use the full file.

**We will grade your homework using `small`. Hidden test 0.0 will override your file selection, so as long as you do not change the file selection in a cell that comes after that one, you will be fine.**

* `linkedin.json` (3M records)
* `linkedin_small.json` (100K records)
* `linkedin_tiny.json` (10K records)

The cell below will download a 3GB file to your Google Cloud. It may take a while. You do not need to modify the two cells below.

In [0]:
url = 'https://upenn-bigdataanalytics.s3.amazonaws.com/linkedin.zip'
filehandle, _ = urllib.request.urlretrieve(url,filename='local.zip')
filehandle = 'local.zip'

In [0]:
# What's the zip file actually called locally?

filehandle

'local.zip'

The cell below creates pointers to the three versions of our dataset. To switch between them, simply change the `file` variable in the cell after the cell below.

In [0]:
def fetch_file(fname):
    zip_file_object = zipfile.ZipFile(filehandle, 'r')
    for file in zip_file_object.namelist():
        file = zip_file_object.open(file)
        if file.name == fname: return file
    return None
    
linkedin_tiny = fetch_file('linkedin_tiny.json')
linkedin_small = fetch_file('linkedin_small.json')
linkedin_huge = fetch_file('linkedin.json')

In [0]:
# CIS 545 Hidden Test 0.0 - please do not modify or delete this cell!

# Set the input file to process
file = linkedin_small


## Step 0.1:  Store data in dataframes

In the cell below, adapt the data loading code from the [in-class notebook](https://colab.research.google.com/drive/1V-QGYHI3YdLVv3sjnfQn7xmy6xEdjJnm).  You will need the function that extracts relations from JSON files and the function that converts relations to dataframes. Read in a maximum of 20000 people. Put the code that reads a line of the file, extracts the relations, removes the interval field, and stores the field information with a try statement, just in case. In the error case, just use a `pass` command to move on. At the end of the next cell, you should have nine dataframes with the following names:

1. `people_df`
2. `names_df`
3. `education_df`
4. `groups_df`
5. `skills_df`
6. `experience_df`
7. `honors_df`
8. `also_view_df`
9. `events_df`

In [0]:
# TODO: Adapt the data loading code from class.
#linked_in = urllib.request.urlopen('https://www.cis.upenn.edu/~cis545/xaa')

START = 0
LIMIT = 20000

def get_df(rel):
    ret = pd.DataFrame(rel).fillna('')
    for k in ret.keys():
        ret[k] = ret[k].astype(str)
    return ret

def extract_relation(rel, name):
    '''
    Pull out a nested list that has a key, and return it as a list
    of dictionaries suitable for treating as a relation / dataframe
    '''
    # We'll return a list
    ret  = []
    if name in rel:
        ret2 = rel.pop(name)
        try:
            # Try to parse the string as a dictionary
            ret2 = json.loads(ret2.replace('\'','\"'))
        except:
            # If we get an error in parsing, we'll leave as a string
            pass
        
        # If it's a dictionary, add it to our return results after
        # adding a key to the parent
        if isinstance(ret2, dict):
            item = ret2
            item['person'] = rel['_id']
            ret.append(item)
        else:
            # If it's a list, iterate over each item
            index = 0
            for r in ret2:
                item = r
                if not isinstance(item, dict):
                    item = {'person': rel['_id'], 'value': item}
                else:
                    item['person'] = rel['_id']
                    
                # A fix to a typo in the data
                if 'affilition' in item:
                    item['affiliation'] = item.pop('affilition')
                    
                item['pos'] = index
                index = index + 1
                ret.append(item)
    return ret
    

names = []
people = []
groups = []
education = []
skills = []
experience = []
honors = []
also_view = []
events = []


#conn = sqlite3.connect('linkedin.db')

lines = []
i = 1
for line in file:
    if i > START + LIMIT:
        break
    elif i >= START:
        person = json.loads(line)

        # By inspection, all of these are nested dictionary or list content
        nam = extract_relation(person, 'name')
        edu = extract_relation(person, 'education')
        grp = extract_relation(person, 'group')
        skl = extract_relation(person, 'skills')
        exp  = extract_relation(person, 'experience')
        hon = extract_relation(person, 'honors')
        als = extract_relation(person, 'also_view')
        eve = extract_relation(person, 'events')
        
        # This doesn't seem relevant and it's the only
        # non-string field that's sometimes null
        if 'interval' in person:
            person.pop('interval')
        
        lines.append(person)
        names = names + nam
        education = education + edu
        groups  = groups + grp
        skills = skills + skl
        experience = experience + exp
        honors = honors + hon
        also_view = also_view + als
        events = events + eve
        
    i = i + 1

people_df = get_df(pd.DataFrame(lines))
names_df = get_df(pd.DataFrame(names))
education_df = get_df(pd.DataFrame(education))
groups_df = get_df(pd.DataFrame(groups))
skills_df = get_df(pd.DataFrame(skills))
experience_df = get_df(pd.DataFrame(experience))
honors_df = get_df(pd.DataFrame(honors))
also_view_df = get_df(pd.DataFrame(also_view))
events_df = get_df(pd.DataFrame(events))

# YOUR CODE HERE
#raise NotImplementedError()

In [0]:
# CIS 545 Hidden Test 0.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 0.1.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 0.1.3 - please do not modify or delete this cell!


## Step 0.2: Convert to SQL

Next save the data to SQLite...  Again, using the same approach as in the sample notebook.

In [0]:
conn = sqlite3.connect('linkedin.db')
people_df.to_sql('people', conn, if_exists='replace', index=False)
names_df.to_sql('names', conn, if_exists='replace', index=False)
education_df.to_sql('education', conn, if_exists='replace', index=False)
groups_df.to_sql('groups', conn, if_exists='replace', index=False)
skills_df.to_sql('skills', conn, if_exists='replace', index=False)
experience_df.to_sql('experience', conn, if_exists='replace', index=False)
honors_df.to_sql('honors', conn, if_exists='replace', index=False)
also_view_df.to_sql('also_view', conn, if_exists='replace', index=False)
events_df.to_sql('events', conn, if_exists='replace', index=False)

# YOUR CODE HERE
#raise NotImplementedError()

In [184]:
# CIS 545 Sanity Check 0.2.1 - please do not modify or delete this cell!

people_df.describe()

,_id,homepage,industry,interests,locality,overview_html,specilities,summary,url
count,20000,20000,20000,20000,20000,20000,20000,20000,20000
unique,20000,464,1125,5734,1797,923,5796,11094,20000
top,in-aabochkaryov,,Information Technology and Services,,San Francisco Bay Area,,,,http://www.linkedin.com/in/aimeekuvadia
freq,1,19537,2103,14232,876,19070,14192,8894,1


In [185]:
# CIS 545 Sanity Check 0.2.2 - please do not modify or delete this cell!

skills_df.describe()
display(experience_df)

,desc,end,org,person,pos,start,title
0,Biomarker Leader for compounds in clinical dev...,Present,Johnson and Johnson,in-00001,0,November 2009,"Senior Scientist, Oncology Biomarkers"
1,Single Cell Gene expression.,,Albert Einstein Medical Center,in-00001,1,September 2008,Associate at Dept of Molecular Genetics
2,Work on peptide to restore wt p53 function in ...,,Columbia University,in-00001,2,August 2006,Associate Research Scientist
3,Molecular profiling of colorectal cancer.,,Memorial Sloan Kettering Cancer Center,in-00001,3,January 2003,Post Doctoral Research Fellow
4,Cancer Research at Dept of Surgery.Molecular p...,,Sahlgrenska University Hospital,in-00001,4,November 2001,Research Scientist
5,,Present,UCSF,in-00006,0,September 2012,Assistant Professor
6,,,Wyss Institute for Biologically Inspired Engin...,in-00006,1,May 2009,Technology Development Fellow
7,Creation of Ning site with integration to back...,Present,<Online Recruiting Company>,in-000montgomery,0,December 2009,Ning
8,.Net 4.0 / C# / VS 2010 / Silverlight 4.0 / Bl...,Present,<Medical Testing Company>,in-000montgomery,1,December 2009,.Net 4.0 / C# / VS 2010 / Silverlight 4.0 / Bl...
9,The new address for Business Intelligence Tool...,Present,000Montgomery.Com,in-000montgomery,2,January 2005,Owner


# Step 1: What is a data scientist?

In this homework, we will use LinkedIn to analyze what it means to be a data scientist (as of a few years ago).

## Step 1.1: What are common skills for data scientists?

Our first question is:  for anyone who's job revolves around data (database administrators, data curators, data engineers, data scientists), *what are the most common skills*?

### Step 1.1.1: Collect skills (Pandas)

Complete the `collect_skills` function below. This and the other functions in this homework allow us to evaluate the correctness of your queries even if your data do not match ours. The function should:

1. Using `experience_df`, find all people with a position containing "data" in the title. Remember upper versus lower case.
2. Using `skills_df`, find all people with "data science" as a skill. Again, remember to account for case.
3. For all of the unique people found in steps 1 and 2, find the rest of their skills
4. Return a dataframe of the top 15 skills, by frequency  (see pandas.DataFrame.sort_values).  The columns should be called `skill` (the name of the skill) and `scientists` (the count of the number of data scientists with this skill).

In [0]:
# TODO: Find the top 15 skills for data scientists (Pandas)
def collect_skills(experience_df, people_df, skills_df):
  df1=experience_df[experience_df['title'].str.contains("data", na=False, case=False)]
  df2=skills_df[skills_df['value'].str.contains("data science", na=False, case=False)]
  lst1=df1['person'].drop_duplicates().tolist()
  lst2=df2['person'].drop_duplicates().tolist()
  lst=list(set(lst1+lst2))
  df=skills_df[skills_df['person'].isin(lst)]
  
  lst1=df['value'].value_counts().index.tolist()[:15]
  lst2=df['value'].value_counts().values.tolist()[:15]
  top_skills_df=pd.DataFrame({'skill':lst1,'scientists':lst2})
  return top_skills_df
    # YOUR CODE HERE
    #raise NotImplementedError()

top_skills_df = collect_skills(experience_df, people_df, skills_df)


def collect_skills(experience_df, people_df, skills_df):
  df1=experience_df[experience_df['title'].str.contains("data", na=False, case=False)]
  df2=skills_df[skills_df['value'].str.contains("data science", na=False, case=False)]
  lst1=df1['person'].drop_duplicates().tolist()
  lst2=df2['person'].drop_duplicates().tolist()
  lst=list(set(lst1+lst2))
  df=skills_df[skills_df['person'].isin(lst)]
  df=df.groupby('value').count().sort_values(by=['person'], ascending = False).reset_index(drop = False)
  df=df.drop(['pos'], axis=1)
  df.columns=['skill','scientists']
  top_skills_df=df[:15]
  return top_skills_df
  #lst1=df['value'].value_counts().index.tolist()[:15]
  #lst2=df['value'].value_counts().values.tolist()[:15]
  #top_skills_df=pd.DataFrame({'skill':lst1,'scientists':lst2})
  
  
  return top_skills_df
    # YOUR CODE HERE
    #raise NotImplementedError()
    
def collect_skills(experience_df, people_df, skills_df):
  pos_data=experience_df[experience_df['title'].str.contains('Data',case=False)][['person']].drop_duplicates()

  #"data science" as a skill
  data_skill_ppl=skills_df[skills_df['value'].str.contains('data science',case=False)][['person']].drop_duplicates()

  #datapeople
  datapeople=pos_data.merge(data_skill_ppl,left_on='person',right_on='person',how='outer')

  #datapeople_skill
  datapeople_skill=datapeople.merge(skills_df[['person','value']],left_on='person',right_on='person',how='left')\
  .groupby('value').count().sort_values(by='person', ascending = False).reset_index(drop = False)


  datapeople_skill.columns=['skill','scientists']
  datapeople_skill15=datapeople_skill[:15]
  return datapeople_skill15


#top_skills_df = collect_skills(experience_df, people_df, skills_df)


In [0]:
# CIS 545 Sanity Check 1.1.1 - please do not modify or delete this cell!

top_skills_df = collect_skills(experience_df, people_df, skills_df)
display(top_skills_df)

if "skill" not in top_skills_df:
    raise AssertionError("skill column not defined")
if "scientists" not in top_skills_df:
    raise AssertionError("scientists column not defined")
if len(top_skills_df) != 15:
    raise AssertionError("dataframe does not have top 15")  

,person
0,in-001monica
1,in-061566
2,in-08012008
3,in-1hansdeboer
4,in-1mereng
5,in-1rajangupta
6,in-39isfine
7,in-3drayfx
8,in-6ix4our
9,in-78697869


,skill,scientists
0,SQL,54
1,Business Intelligence,50
2,Data Warehousing,43
3,Project Management,41
4,Data Analysis,41
5,Business Analysis,35
6,Databases,33
7,Microsoft Office,33
8,Java,33
9,SQL Server,32


In [0]:
# CIS 545 Hidden Test 1.1.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.1.1.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.1.1.3 - please do not modify or delete this cell!


### Step 1.1.2: Top skills (SQL)

Compute the same table as in 1.1.1 using SQL. Store it as `top_skills_sql` but otherwise matching the schema and other properties. Be sure to also save the data to SQLLite in a table called `top_skills`, as we will be testing to see if this table exists.

In [0]:
# TODO: Find the top 15 skills for data scientists (SQL)

# YOUR CODE HERE
#raise NotImplementedError()


# new Table for unique people
conn.execute('''
CREATE TABLE UniquePeopleA (
person TEXT
)''')

conn.execute('''
CREATE TABLE UniquePeopleB (
person TEXT
)''')

#select people with skill, insert into new table
conn.execute('''
INSERT INTO UniquePeopleA (person)
SELECT person FROM skills
WHERE value LIKE 'data science'
''')

#select people with experience, insert into new table
conn.execute('''
INSERT INTO UniquePeopleB (person)
SELECT person FROM experience
WHERE title LIKE '%data%'
''')

#merge
conn.execute('''CREATE TABLE dataPeople AS
SELECT person
FROM UniquePeopleA
LEFT JOIN UniquePeopleB USING(person)
UNION ALL
SELECT person
FROM UniquePeopleB
LEFT JOIN UniquePeopleA USING(person)
WHERE UniquePeopleA.person IS NULL
GROUP BY person

''')

#find skills
conn.execute('''
CREATE TABLE dataPeopleSkill AS
SELECT skills.person,skills.value
FROM skills
INNER JOIN dataPeople ON dataPeople.person=skills.person
''')

#limit&store

conn.execute('''
CREATE TABLE top_skills_sql AS
SELECT count(person) as scientists, value as skill from dataPeopleSkill
group by skill
order by scientists desc
LIMIT 15
''')

top_skills_sql = pd.read_sql('SELECT * from top_skills_sql', conn)
display(top_skills_sql)

OperationalError: ignored

In [0]:
# CIS 545 Sanity Check 1.1.2 - please do not modify or delete this cell!

if "skill" not in top_skills_sql:
    raise AssertionError("skill column not defined")
if "scientists" not in top_skills_sql:
    raise AssertionError("scientists column not defined")
if len(top_skills_df) < 1:
    raise AssertionError("dataframe has no results")  
if len(top_skills_sql.merge(top_skills_df)) != len(top_skills_sql):
    raise AssertionError("Pandas and SQL versions are not of the same length")

In [0]:
# CIS 545 Hidden Test 1.1.2 - please do not modify or delete this cell!


## Step 1.2: What are common titles for those with data science skills?

Complete the `collect_titles` function below that aggregates the most recent titles of people with data science skills. This function should use the given dataframes as input and return a two column dataframe: one column called `title` and the other called `count`. You should only consider people who have at least `min_skills` of the top skills for a data scientist. You should also only keep those titles that appear at least `min_count` times.

For extra practice, you can also do this in SQL, although we are not grading that.

In [0]:
def collect_titles(top_skills_df, skills_df, people_df, experience_df, min_skills, min_count):

#find people who have skills in top_skills, group skill, count times
  top_skills_ppl_df=top_skills_df.merge(skills_df, left_on='skill', right_on='value', how='inner')\
  [['person',"skill"]]\
  .groupby(by='person')\
  .count()\
  .sort_values(by=['skill'],ascending=False).reset_index()

#if greater than min_skills(use6), pull out people
  top_skills_ppl_small=top_skills_ppl_df.loc[(top_skills_ppl_df['skill'] >=min_skills )]

#get these people's Most Recent position in df
  mostRecent_df=experience_df.loc[(experience_df['pos'] =="0")]

#find these everyone's recent title
  titleSort_df=top_skills_ppl_small.merge(mostRecent_df, left_on='person', right_on='person', how='inner')\
  [['person','title']]\
  .groupby(by='title').count().sort_values(by=['person'],ascending=False)\
  .reset_index()

#if count>=min_count(2), pull out
  collect_titles=titleSort_df.loc[(titleSort_df['person'] >=min_count)]
  collect_titles.columns=["title",'count']
  return collect_titles


# TODO: Find the common titles (Pandas)

def collect_titles(top_skills_df, skills_df, people_df, experience_df, min_skills, min_count):
    # YOUR CODE HERE
    #raise NotImplementedError()
    lst=top_skills_df['skill'].tolist()
    df=skills_df[skills_df['value'].isin(lst)]
    #display(df['person'].value_counts())
    lst1=df['person'].value_counts().index.tolist()
    lst2=df['person'].value_counts().values.tolist()
    df=pd.DataFrame({'name':lst1,'count':lst2})
    df=df[df['count']>=min_skills]
    lst=df['name'].tolist()
    recent_df=experience_df[experience_df['pos']=='0']
    df_t0=recent_df[recent_df['person'].isin(lst)]
    lst1=df_t0['title'].value_counts().index.tolist()
    lst2=df_t0['title'].value_counts().values.tolist()
    ds_titles_df=pd.DataFrame({'title':lst1,'count':lst2})
    ds_titles_df=ds_titles_df[ds_titles_df['count']>=min_count]
    #display(df_t)

    
    return ds_titles_df

ds_titles_df = collect_titles(top_skills_df, skills_df, people_df, experience_df, 6, 2)

In [0]:
# CIS 545 Sanity Check 1.2 - please do not modify or delete this cell!

ds_titles_df = collect_titles(top_skills_df, skills_df, people_df, experience_df, 6, 2)
display(ds_titles_df)

if "title" not in ds_titles_df:
    raise AssertionError("title column not defined")
if "count" not in ds_titles_df:
    raise AssertionError("count column not defined")
if len(ds_titles_df) < 1:
    raise AssertionError("dataframe has no results")

,title,count
0,Consultant,3
1,Software Engineer,3
2,Senior Software Engineer,3
3,Software Architect,2
4,Web Developer,2
5,Software Developer,2


In [0]:
# CIS 545 Hidden Test 1.2.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.2.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.2.3 - please do not modify or delete this cell!


## Step 1.3: Who employs "data people" based on title?

Now let's find the list of companies that have employed people with the above titles, ranked by number of employees who have had these titles.

### Step 1.3.1: Data employers

Complete the `collect_employers` function below that aggregates the employers with positions corresponding to the most recent titles of people with data science skills. This function should use the given dataframes as input and return a two column dataframe: one column called `org` and the other called `people`. Show the names of companies (in field `org`) with at least `min_count` employees who are "data people" (include that count in the `people` column). Order the dataframe by the count of data people in the company in descending order.

In [0]:
# TODO: Find the data employers

def collect_employers(experience_df, ds_titles_df, min_count):
    lst=top_skills_df['skill'].tolist()
    df=skills_df[skills_df['value'].isin(lst)]
    lst=ds_titles_df['title'].tolist()
    df_1=experience_df[experience_df['title'].isin(lst)]
    df_1=df_1[['org','person']]
    lst1=df_1['org'].value_counts().index.tolist()
    lst2=df_1['org'].value_counts().values.tolist()
    employers_df=pd.DataFrame({'org':lst1,'people':lst2})
    employers_df=employers_df[employers_df['people']>=min_count]
    return employers_df
    # YOUR CODE HERE
    #raise NotImplementedError()
    
def collect_employers(experience_df, ds_titles_df, min_count):
#turn df into sql
  ds_titles_df.to_sql('ds_titles_sql_13', conn, if_exists='replace', index=False)
  experience_df.to_sql('experience_sql_13', conn, if_exists='replace', index=False)

#get position and corresponding org  
  exp_orgTitle_df=pd.read_sql('''
    select title,org from experience_sql_13
  ''',conn)

#get most recent pos of datapeople
  datapeople_title_13_df = pd.read_sql('''
    select title from ds_titles_sql_13
    ''', conn)
  
#get desired employers
  collect_employers_non=datapeople_title_13_df.merge(exp_orgTitle_df, left_on='title', right_on='title', how='inner')\
  .rename(columns={"title": "people"})\
  .groupby(by='org')\
  .count()\
  .sort_values(by=['people'],ascending=False).reset_index()
  
  collect_employers=collect_employers_non.loc[(collect_employers_non['people'] >=min_count)]

  return collect_employers


In [0]:
# CIS 545 Sanity Check 1.3.1 - please do not modify or delete this cell!

employers_df = collect_employers(experience_df, ds_titles_df, 5)
display(employers_df)

if "IBM" not in employers_df['org'].tolist():
    raise AssertionError("Missing IBM")
    
if employers_df['people'].min() < 4:
    raise AssertionError("Not filtering properly")

,org,people
0,Google,28
1,Accenture,24
2,Infosys,17
3,Infosys Technologies Ltd,11
4,Capgemini,10
5,IBM,10
6,Freelance,9
7,The Boston Consulting Group,9
8,Sun Microsystems,9
9,LinkedIn,8


In [0]:
# CIS 545 Hidden Test 1.3.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.3.1.2 - please do not modify or delete this cell!


### Step 1.3.2:  Their employees

Complete the `collect_employees` function below that aggregates the employees of employers with positions corresponding to the most recent titles of people with data science skills. In other words, who are the employees of the data employers you found before and what are their titles? This function should use the given dataframes as input and return the `org`, `family_name`, `given_name`, and `title` of each person.

In [0]:
# YOUR CODE HERE

#title_people_df=pd.DataFrame({'org':lst1,'family_name':lst_f,'given_name':lst_g,'title':lst2})

def collect_employees(people_df, experience_df, employers_df, names_df, ds_titles_df):
#experience_df132 is what pos=0 in experience_df
  conn.execute(''' DROP TABLE IF EXISTS experience_132 ''')
  conn.execute(''' DROP TABLE IF EXISTS tempTbale1321 ''')
  experience_df.to_sql('experience_132', conn, if_exists='replace', index=False)
  conn.execute('''
  CREATE TABLE tempTbale1321 AS
  SELECT org,person,title
  FROM experience_132
  WHERE pos="0"
  ''')

  experience_df132 = pd.read_sql('''
    select * from tempTbale1321
    ''', conn)


  test_df = names_df[['person','family_name','given_name']]\
  .merge(experience_df132[['org','person','title']] , left_on=['person'] , right_on=['person'] , how='inner')\
  [['org','family_name','given_name','title']]\
  .merge(employers_df[['org']] , left_on=['org'] , right_on=['org'] , how='inner')

  return test_df


# TODO: Find the employees of the data employers
def collect_employees(people_df, experience_df, employers_df, names_df, ds_titles_df):
    #raise NotImplementedError()
    recent_df=experience_df[experience_df['pos']=='0']
    lst=ds_titles_df['title'].tolist()
    df_1=experience_df[experience_df['title'].isin(lst)]
    df_1=df_1[['org','person','title']]
    title_people_df=df_1.merge(names_df,left_on='person',right_on='person', how='inner')
    lst=employers_df['org'].tolist()
    #df=recent_df[recent_df['value'].isin(lst)]
    title_people_df=title_people_df[title_people_df['org'].isin(lst)]
    return title_people_df[['org','family_name','given_name','title']]




In [0]:
# CIS 545 Sanity Check 1.3.2 - please do not modify or delete this cell!

title_people_df = collect_employees(people_df, experience_df, employers_df, names_df, ds_titles_df)
display(title_people_df)

if len(title_people_df.columns) != 4:
    raise AssertionError('Wrong number of columns. Check schema again')

,org,family_name,given_name,title
29,Capgemini,Ibrahim,Mohamed,Consultant
34,Motorola,Laipple,John,Software Engineer
46,Accenture,Pani,Michele,Consultant
81,Oracle,Sood,Aaditya,Software Engineer
123,IBM,Bento,Aaron,Software Engineer
127,Google,Hopkins,Aaron,Software Engineer
151,Accenture,Lilly,Aaron,Consultant
164,Freelance,Ow,Aaron,Senior Software Engineer
177,Deloitte,Suh,Aaron,Consultant
188,Accenture,Joshi,Aashish,Software Engineer


In [0]:
# CIS 545 Hidden Test 1.3.2.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.3.2.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.3.2.3 - please do not modify or delete this cell!


## Step 1.4: Find peers

In many common social graph settings, we can make recommendations to people based on their similarity with other people. In this case, we define similarity in terms of the number of identical skills.

Suppose A and B have similar skills: A -> X1 and B -> X1, A -> X2 and B -> X2, etc. up to A -> Xk and B -> Xk.

Then given that A and B have similar skills, we might recommend A's employer to B, and vice versa.

### Step 1.4.0: Making the problem tractable in Pandas

Let's consider only the first 100 people in `people_df`.

Find, out of this set, the pairs of people with the most shared/common skills, and return the closest 20 pairs in descending order.  We'll then use this to make a *recommendation* for a potential employer and position to each person.

### Step 1.4.1: Compute the top pairs of peers

Complete the `collect_peers` function below that finds the top `num` pairs of peers. In other words, compare each person with each *other* person, counting the total set of skills in common. This function should use the given dataframes and `num` as input and return a three column dataframe: `person_1`, `person_2`, and `common_skills`. The first two columns should be person IDs and the last column should be the number of skills that this pair of people shares.

Hint: Doing this requires a *Cartesian product*, i.e., every ID paired with every other ID.  Think about how to create a dataframe just with people IDs, then add a field to this dataframe that will let us combine every record with every record.

In [0]:
# TODO: Finish the collect_peers function

people_df_subset = people_df.head(100)
lst=people_df_subset['_id'].tolist()
df_1=skills_df[['person']].merge(people_df_subset[['_id']],left_on='person',right_on='_id',)
df_1=skills_df[skills_df['person'].isin(lst)]
#df_1=df_1.groupby(by=['person']).count()
display(df_1)
#print(len(df_1['person'].drop_duplicates().tolist()))

'''
people_df_subset['trivial']=0
cur=people_df_subset[['_id','trivial']]
cur=cur.merge(cur, left_on='trivial',right_on='trivial').drop(['trivial'],axis=1)
cur=cur[cur['_id_x'] != cur['_id_y']]
lst1=cur['_id_x'].tolist()
lst2=cur['_id_y'].tolist()


import numpy as np
res=np.zeros([len(lst1),2])
for i in range(len(lst1)):
  a=set(df_1.loc[lst1[i]].index.tolist())
  b=set(df_1.loc[lst2[i]].index.tolist())
  
  res[i][0]=len(a.union(b))
  res[i][1]=len(a.union(b))
res.sort(reverse=True)
print(res[:20])
def collect_peers(people_df_subset, skills_df, num):
  
    # YOUR CODE HERE
    #raise NotImplementedError()
    pass
'''

,person,pos,value
0,in-00000001,0,Key Account Development
1,in-00000001,1,Strategic Planning
2,in-00000001,2,Market Planning
3,in-00000001,3,Team Leadership
4,in-00000001,4,Negotiation
5,in-00000001,5,Forecasting
6,in-00000001,6,Key Account Management
7,in-00000001,7,Sales Management
8,in-00000001,8,New Business Development
9,in-00000001,9,Business Planning


70


"\npeople_df_subset['trivial']=0\ncur=people_df_subset[['_id','trivial']]\ncur=cur.merge(cur, left_on='trivial',right_on='trivial').drop(['trivial'],axis=1)\ncur=cur[cur['_id_x'] != cur['_id_y']]\nlst1=cur['_id_x'].tolist()\nlst2=cur['_id_y'].tolist()\n\n\nimport numpy as np\nres=np.zeros([len(lst1),2])\nfor i in range(len(lst1)):\n  a=set(df_1.loc[lst1[i]].index.tolist())\n  b=set(df_1.loc[lst2[i]].index.tolist())\n  \n  res[i][0]=len(a.union(b))\n  res[i][1]=len(a.union(b))\nres.sort(reverse=True)\nprint(res[:20])\ndef collect_peers(people_df_subset, skills_df, num):\n  \n    # YOUR CODE HERE\n    #raise NotImplementedError()\n    pass\n"

In [0]:
# CIS 545 Sanity Check 1.4.1 - please do not modify or delete this cell!

recs_df = collect_peers(people_df_subset, skills_df, 20)
display(recs_df)

if "person_1" not in recs_df:
    raise AssertionError("person_1 column not defined")
if "person_2" not in recs_df:
    raise AssertionError("person_2 column not defined")
if "common_skills" not in recs_df:
    raise AssertionError("common_skills column not defined")
if(len(recs_df) != 20):
    raise AssertionError('Wrong number of rows in recs_df')

In [0]:
# CIS 545 Hidden Test 1.4.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.4.1.2 - please do not modify or delete this cell!


### Step 1.4.2: Get the last jobs

Complete the `last_job` function below that takes `experience_df` as input and returns the `person`, `title`, and `org` corresponding to each person's **last** (most recent) employment experience (three column dataframe).

In [0]:
# TODO: Complete the last_job function

def last_job(experience_df):
  
  last_job_df=experience_df[experience_df['pos']=='0']
  last_job_df=last_job_df[['person','title','org']]
  return last_job_df
    # YOUR CODE HERE
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 1.4.2 - please do not modify or delete this cell!

last_job_df = last_job(experience_df)
display(last_job_df)

if(len(last_job_df.columns) != 3):
    raise AssertionError('Wrong number of columns in last_job_df')

,person,title,org
0,in-00001,"Senior Scientist, Oncology Biomarkers",Johnson and Johnson
5,in-00006,Assistant Professor,UCSF
7,in-000montgomery,Ning,<Online Recruiting Company>
36,in-001adambutler,Founding Partner and Client Services Director,Brand New Music
46,in-001monica,"Manager, Marketing",Canadian MedicAlert Foundation
49,in-001neilpeacock,Sales Manager - Security Team,Complete IT Systems Ltd
55,in-00789123,Price Manager,Ericsson Regional Sub-Saharan Africa
59,in-007katarzynagola,Project Manager,K2 Search
66,in-00911,Zaakvoerster,I & I Recrutering en selectie BVBA
71,in-00danielnewman00,CEO/CTO,Convozine


In [0]:
# CIS 545 Hidden Test 1.4.2.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.4.2.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.4.2.3 - please do not modify or delete this cell!


### Step 1.4.3: Recommend jobs

Complete the `recommend_jobs` function below that takes `recs_df`, `names_df`, and `last_job_df` as input and returns for each `person_1`, `person_2`'s most recent `title` and `org`.

In [0]:
# TODO: Complete the recommend_jobs function

def recommend_jobs(recs_df, names_df, last_job_df):
  
    # YOUR CODE HERE
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 1.4.3 - please do not modify or delete this cell!

recommended_df = recommend_jobs(recs_df, names_df, last_job_df)
display(recommended_df)

if "family_name" not in recommended_df:
    raise AssertionError("person_1 column not defined")
if "given_name" not in recommended_df:
    raise AssertionError("person_2 column not defined")
if "person_2" not in recommended_df:
    raise AssertionError("common_skills column not defined")
if "org" not in recommended_df:
    raise AssertionError("common_skills column not defined")
if "title" not in recommended_df:
    raise AssertionError("common_skills column not defined")

In [0]:
# CIS 545 Hidden Test 1.4.3 - please do not modify or delete this cell!


# Step 2: Compare Evaluation Orders

This last section relates to our discussions in lecture about computation efficiency with big data.

## Step 2.0: Load custom functions

Let's look at some computation and optimization tasks.  We'll start with the code from our lecture notebooks, which does joins between dataframes.

In [0]:
# Join using nested loops
def merge(S,T,l_on,r_on):
    ret = pd.DataFrame()
    count = 0
    S_ = S.reset_index().drop(columns=['index'])
    T_ = T.reset_index().drop(columns=['index'])
    for s_index in range(0, len(S)):
        for t_index in range(0, len(T)):
            count = count + 1
            if S_.loc[s_index, l_on] == T_.loc[t_index, r_on]:
                ret = ret.append(S_.loc[s_index].append(T_.loc[t_index].drop(labels=r_on)), ignore_index=True)

    print('Merge compared %d tuples'%count)
    return ret
  
# Join using a *map*, which is a kind of in-memory index
# from keys to (single) values
def merge_map(S,T,l_on,r_on):
    ret = pd.DataFrame()
    T_map = {}
    count = 0
    # Take each value in the r_on field, and
    # make a map entry for it
    T_ = T.reset_index().drop(columns=['index'])
    for t_index in range(0, len(T)):
        # Make sure we aren't overwriting an entry!
        assert (T_.loc[t_index,r_on] not in T_map)
        T_map[T_.loc[t_index,r_on]] = T_.loc[t_index]
        count = count + 1

    # Now find matches
    S_ = S.reset_index().drop(columns=['index'])
    for s_index in range(0, len(S)):
        count = count + 1
        if S_.loc[s_index, l_on] in T_map:
                ret = ret.append(S_.loc[s_index].append(T_map[S_.loc[s_index, l_on]].drop(labels=r_on)), ignore_index=True)

    print('Merge compared %d tuples'%count)
    return ret8

## Step 2.1: Find an optimal order of evaluation.

Reimplement `recommend_jobs` using the above `merge` or `merge_map` functions instead of Pandas' merge. Try to find the **most efficient** way.  You should start with the dataframes `recs_df`, `names_df`, and `last_job_df` from above. Store your results in `recs_new_df`

In [0]:
# TODO: Reimplement recommend jobs using our custom merge and merge_map functions

def recommend_jobs_new(recs_df, names_df, last_job_df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 2.1 - please do not modify or delete this cell!

%%time

recs_new_df = recommend_jobs_new(recs_df, names_df, last_job_df)

if(len(recs_new_df.columns) != 5):
    raise AssertionError('Wrong number of columns in recs_new_df')

# Step 3: Submitting Your Homework

1. When you are done, select “Edit” at the top of the window, **under the filename, not the one that may appear above it**. Then, select “Clear all outputs”. Please do this just before turning is your homework because it reduces the size of your file.


2. In the same menu **under the filename**, select “File” and then “Download .ipynb”. It is very important that you do not change the file name of this downloaded notebook. Make sure that something like “(1)” did not get added to the filename and also that you did not download the .py version. Our autograder can only handle .ipynb files with the correct file name.

3. Compress the ipynb file into a Zip file **hw2.zip**.

4. Go to the [submission site](http://submit.dataanalytics.education), and click on the Google icon.  Log in using your Google@SEAS (if at all possible!) or (if you aren’t an Engineering student) GMail account.  

5. Click on the **Courses** icon at the top, then select **CIS 545** and **Save**. Select **cis545-2019c-hw2** and upload **hw2.zip**.

6. You should see a message on the submission site notifying you about whether your submission passed validation.  You may resubmit as necessary, but may have to withdraw your previous submission in OpenSubmit in order to do so.

**If you have not already, please go to Settings and set your Student ID to your PennID (all numbers)**.